# Finite fields

Some python for

* Fields $\mathbb{Z}_p$ for prime $p$.
* The polynomial rings over a field


In [1]:
from IPython.display import display, Math, Latex
import functools

## Fields $\mathbb{Z}_p$

Now define a class for fields $\mathbb{Z}_p$. The goal is to support polynomial rings over any field, so we take some care to implement $\mathbb{Z}_p$ using a uniform interface. 

First, we need the Euclid/Bezout algorithm for computing multiplicative inverses.
There are much more efficient methods if we compute in base 2, but the reason I chose this version will be clear later in the notebook.

In [2]:
def bezout(m,n):
    s,t,u,v = 1,0,0,1
    while n != 0:
        q = m//n
        m, n = n, m%n
        s, t = t, s - q*t
        u, v = v, u - q*v
    return s, u

# Multiplicative inverse of m modulo p (assuming they are relatively prime)
def inverse(m,p):
    return bezout(m,p)[0] % p

In [3]:
a = inverse(2, 31)
a

16

In [4]:
(12*inverse(12,31))%31

1

Define the fields $\mathbb{Z}_p$ for prime $p$.

In this implementation is _field_ is just a class that has the following standard operations defined:

* A constructor that takes integer $n$ to $1+1+\ldots + 1$ in the field (remember these are elements of the _prime subfield_)
* The four arithmetic operations
* == and !=
* string and latex string representations

In [5]:
def Zp(p):
    inv_table = {k:inverse(k,p) for k in range(1,p)}
    
    class field_elmt:
        def __init__(this,a):
            if type(a) == int:
                this.val = a%p
            else:
                this.val = a.val
    
        def inv(this,a):
            return inv_table[a]
        
        def __eq__(this, b):
            return this.val == b.val
        
        def __neq__(this,b):
            return this.val != b.val

        def __add__(this,b):
            return field_elmt(this.val+b.val)

        def __mul__(this,b):
            return field_elmt(this.val*b.val)

        def __sub__(this,b):
            return field_elmt(this.val-b.val)

        def __truediv__(this,b): # Implements /, not //
            return field_elmt(this.val*inv_table[b.val])

        def _repr(this):
            return '%d' % this.val
        
        def _repr_latex_(this):
            return '$%d$' % this.val
        
    return field_elmt

Let's try it out

In [6]:
Z11 = Zp(11)
Z11(13)/Z11(3)  # Should yield a bit of latex type setting to yield argument mod 11

<__main__.Zp.<locals>.field_elmt at 0x1065a58d0>

In [7]:
Z11(8)*Z11(3)

<__main__.Zp.<locals>.field_elmt at 0x1065a55f8>

We can use arithmetic on data of type Z11:

In [8]:
functools.reduce(lambda m,n: m+n, (Z11(3),Z11(4),Z11(9),Z11(5)),Z11(0))

<__main__.Zp.<locals>.field_elmt at 0x1065a5f60>

A helper for latex. Produces $x^2$, $x^3$, and so on.

In [9]:
def x_expr(i, x_name):
    if i == 0:
        return ''
    elif i == 1:
        return x_name
    else:
        return x_name + '^{' + str(i) + '}'

Produce a class of polynomials over a field $F$.

I've taken some care to support $\LaTeX$ representations of polynomials.

This supports standard operations, also

* $p \mapsto px$
* $px \mapsto px + c$
* $p[i]$ -- extract a coefficient
* Modular division
*  $\text{root}^2$ -- I'll explain this later

In [10]:
def PolyRing(F):
    class polynomial:
        def __init__(self, coeffs):
            if type(coeffs) == int: # only one coefficient
                if coeffs == 0:
                    self.coeffs = ()
                    self.deg = -1
                else:
                    self.coeffs = (F(coeffs),)
                    self.deg = 0
            else:
                coeffs = tuple(coeffs)
                k=len(coeffs)-1
                while k >= 0:
                    if coeffs[k] != F(0):
                        break
                    k -= 1
                self.coeffs = tuple(F(coeffs[j]) for j in range(k+1))
                self.deg = len(self.coeffs)-1
            self.F = F
            
        def __eq__(self,b):
            return self.coeffs == b.coeffs
        
        def __getitem__(self,i):
            if 0 <= i <= self.deg:
                return self.coeffs[i]
            else:
                return F(0)
            
        def __add__(self, b):
            return polynomial((self[i]+b[i] for i in range(max(self.deg, b.deg)+1)))
        
        def __mul__(self,b):
            return polynomial(functools.reduce(F.__add__, (self[i-k]*b[k] for k in range(i+1)),F(0)) for i in range(self.deg+b.deg+1))

        def __sub__(self,b):
            return polynomial((self[i]-b[i] for i in range(max(self.deg, b.deg)+1)))
        
        
        def fulldiv(self,b):
            rem = self
            quot = polynomial(())
            b_lead = b[b.deg]
            k = rem.deg
            while k >= b.deg:
                q = rem[k]/b_lead
                quot = polynomial((q,)+quot.coeffs)
                rem = rem - b.scale(q).shift(k-b.deg)
                k -= 1
            return quot, rem
            
        def __len__(self):
            return self.deg + 1
        
        def __floordiv__(self,b):
            return self.fulldiv(b)[0]
        
        def __mod__(self,b):
            return self.fulldiv(b)[1]
        
        def scale(self, a):
            return polynomial(map(lambda m: a*m,self.coeffs))
            
        def shift(self, n):
            return polynomial((self[i-n] for i in range(self.deg + n + 1)))
        
        def root_sqr(self):
            return polynomial((F(0)-self[i] for i in range(self.deg)))
                        
        # Latex presentation 
        def lead_term(self,x_name):
            if self.deg == 0:
                return self[self.deg]._repr()
            
            if self[self.deg] == F(1):
                c = ''
            else:
                c = self[self.deg]._repr()
            return c + x_expr(self.deg,x_name)
        
        def nxt_term(self, i, x_name):
                
            if self[i] == F(0):
                s = ''
            else:
                s = '+'
                if i==0 or self[i] != F(1):
                    s += self[i]._repr()
                s +=  x_expr(i, x_name)
            return s
            
        def latex(self, x_name):
            if self.deg == -1:
                s = r'0'
            elif self.deg == 0:
                s = self.coeffs[0]._repr()
            else:
                d = self.deg
                s = self.lead_term(x_name)
                
                d -= 1
                while d >= 0:
                    s += self.nxt_term(d, x_name)
                    d -= 1
            return s
        
        def _repr_latex_(self):
            return '$%s$' % self.latex('x')
        
    return polynomial
    

Try things

In [11]:
Z11_x = PolyRing(Z11)

In [12]:
p = Z11_x(map(Z11, (1,2,3,4,5)))

In [21]:
(p*p)//p

<__main__.PolyRing.<locals>.polynomial at 0x1065a20b8>

## Irreducible quadratic polynomials

Make a table products of all monic $1^{\text{st}}$ degree polynomials reduced to monic form, since all $2^{\text{nd}}$ degree monic non-irreducible polynomials will appear in the table.

In [14]:
m = {(a,b):(Z11_x(map(Z11,(a,1))) * Z11_x(map(Z11,(b,1))))# // Z11_x((Z11(b)*Z11(d),)) 
     for a in range(11) for b in range(a+1)}

In [15]:
len(m)

66

In [16]:
display(Latex(r"$(a,b): (x+a)\times (x+b)$"))
for a in range(11):
    for b in range(a+1):
        display(Math('(%d,%d):'%(a,b) + m[(a,b)].latex('x')))

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Make a set of the non-trivial coefficients of each reducible $2^{\text{nd}}$ degree polynomial.

In [17]:
red = set([(p[0].val, p[1].val) for p in m.values()])

In [18]:
len(red)

66

We can complement this set to obtain the non-trivial coefficients for polynomials that include all _irreducibles_. But these may still be divisible by constant. 

In [19]:
irred1 = [Z11_x(map(Z11,(i,j,1))) for i in range(11) for j in range(11) if (i,j) not in red]

In [20]:
print(len(irred1))
for i in range(len(irred1)):
    display(Math(irred1[i].latex('x')))

55


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now we pick an irreducible (anything on the list)

In [22]:
modulus = irred1[1]
modulus

<__main__.PolyRing.<locals>.polynomial at 0x1065d6048>

Multiplication in $\textsf{GF}(p^2)$ operates on $1^{\text{st}}$ degree polynomials by $f\star g = (f\times g) \bmod p$  

In [23]:
def prod(f,g):
    return (f*g)%modulus

A simple sanity check:

In [24]:
f = Z11_x(map(Z11,(4,2)))
g = Z11_x(map(Z11,(3,5)))
h = Z11_x(map(Z11,(10,8)))

In [25]:
prod(f, g+h)

<__main__.PolyRing.<locals>.polynomial at 0x1065ec860>

In [26]:
prod(f,g) + prod(f,h)

<__main__.PolyRing.<locals>.polynomial at 0x1065d63c8>

We can build a new field from an irreducible polynomial $m$.

Elements  $bx+a$ of $GF(p^2)$ are represented as pairs $(a,b)$. Then addition is just polynomial addition.
Multiplication is $(a,b)*(c,d) = (e,f)$ where $fx+e = ((bx+a)(dx+c))\%m$.

But what do we do about division?

It turns out that Bezout works in any ring.

In [27]:
def bezoutTryout(m,n,R):
    s,t,u,v = R(1),R(0),R(0),R(1)
    while n != R(0):
        q = m//n
        m, n = n, m%n
        s, t = t, s - q*t
        u, v = v, u - q*v
    return s, u, m

This almost does what we need. Try it out to see what goes wrong.

In [32]:
for a in range(11):
    for b in range(1,11):
        f = Z11_x((map(Z11,(a,b))))
        (s,u,g)  = bezoutTryout(f,modulus, Z11_x)
        display(Math(('(%dx+%d)\\times(%s) = '%(b,a,s.latex('x'))+((s*f)%modulus).latex('x'))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Try a more subtle approach. Noticing that $s\times f \bmod m$ is always is a constant polynomial, we can divide $s$ by it to get the actual multiplicative inverse of $f$

In [33]:
for a in range(11):
    for b in range(1,11):
        f = Z11_x((map(Z11,(a,b))))
        (s,u,g)  = bezoutTryout(f,modulus, Z11_x)
        display(Math(('(%dx+%d)\\times(%dx+%d)^{-1} = '%(b,a,b,a))+((f*(s//((s*f)%modulus)))%modulus).latex('x')))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now we know how to compute inverses.

In [34]:
def inverseGF(m,n,R):
    s,t,u,v = R(1),R(0),R(0),R(1)
    oldm, oldn = m,n
    while n != R(0):
        q = m//n
        m, n = n, m%n
        s, t = t, s - q*t
        u, v = v, u - q*v
   # display(Math(s.latex('x')))
    return s//((s*oldm)%oldn)

In [35]:
m_l = modulus.latex('x')
for a in range(11):
    for b in range(1,11):
        f = Z11_x((map(Z11,(a,b))))
        f_inv  = inverseGF(f,modulus, Z11_x)
        f_l = f.latex('x')
        f_i_l = f_inv.latex('x')
        p_l = ((f * f_inv)%modulus).latex('x')
        display(Latex('$(%s)^{-1} = %s$. So $(%s)\\times (%s)\\bmod %s = %s.$'%\
                     (f_l , f_i_l, f_l, f_i_l, m_l, p_l)))
        

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

So we have a representation of $GF(11^2)$ by choosing any of our irreducible polynomials. Another approach is to find _primitive_ polynomials instead. These are  essentially irreducible polynomials that also support a usable logarithm table.

Using our example irreducible polynomial $x^2+x^1$, a root of that polynomial would be some $\rho$ so that $0=\rho^2 + \rho + 1$. Or (in the form we will need it), 
$$\rho^2 =   -\rho -\rho = 10\rho + 10$$

Using this identity, we can compute $$\rho^3 = \rho^2\rho = 10\rho^2 + 10\rho,$$ but in our multiplication modulo $x^2 + x + 1$, this is actually $1$. 

So something is wrong.


In [38]:
for i in range(121):
    display(Math(a[i].latex('x')))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

So $x^2 + x  + 1$ does not produce a good logarithm table. We need another modulus.

In [83]:
modulus = irred1[6]
exp = {}
exp[0] = Z11_x((Z11(1),Z11(0))) # this is the polynomial rho^0 = 1
exp[1] = Z11_x((Z11(0),Z11(1))) #                        rho
exp[2] = modulus.root_sqr()     #                        rho^2
for i in range(3,121):        #                        rho^{i+1} = rho*rho^i
    exp[i] = prod(exp[i-1],exp[1])
    
log = {}

for i in range(120):
    log[(exp[i][0].val,exp[i][1].val) ] = i 

In [84]:
log[(8,3)]

3

In [85]:
for i in range(121):
    display(Latex('$x^{%d} = %s$' % (i,exp[i].latex(r'x'))))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Groovy. So our primitive polynomial is

In [86]:
modulus = irred1[6]
modulus

<__main__.PolyRing.<locals>.polynomial at 0x1065d6a20>

In [87]:
def prod2(a,b):
    if a == Z11_x(0) or b == Z11_x(0):
        return Z11_x(0)
    else:
        return exp[log[(a[0].val,a[1].val)] + log[(b[0].val,b[1].val)] % 120 ]
    

In [89]:

prod2((Z11_x(0)),f)

<__main__.PolyRing.<locals>.polynomial at 0x1066b2e48>

In [82]:
log[(1,0)]

120